In [1]:
#基于用户的协同过滤算法
import os
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sbn
import pandas as pd
#正常显示中文标签和负号
#plt.rcParams['font.sans-serif'] = ['SimHei']  #用来正常显示中文标签
#plt.rcParams['axes.unicode_minus'] = False  #用来正常显示负号
import gc

In [2]:
#输入文件
DATA_PATH = os.path.join(os.getcwd(),'data','input')
TRAIN = os.path.join(DATA_PATH,'train.csv')
TEST = os.path.join(DATA_PATH,'test.csv')
SONG =  os.path.join(DATA_PATH,'song.csv')
MEMBER =  os.path.join(DATA_PATH,'member.csv')

In [3]:
#读取文件
train = pd.read_csv(TRAIN)
test = pd.read_csv(TEST)
#member = pd.read_csv(MEMBER)
#song = pd.read_csv(SONG)
train = train[['msno','song_id','target']]
test = test[['msno','song_id','target']]

In [4]:
#计算用户和电影数量
msno_set = set(train['msno'])|set(test['msno'])
song_set = set(train['song_id'])|set(test['song_id'])
n_users = len(msno_set)
n_items = len(song_set)

In [8]:
#train.head()

,msno,song_id,target
0,16135,24598,1
1,13696,9154,0
2,187,8457,0
3,7843,28248,1
4,4527,12389,1


In [5]:
#创建用户——音乐矩阵
#因为又有测试数据，又有训练数据需要创建两个矩阵
train_data_matrix = np.zeros((n_users,n_items))
for line in train.itertuples():  #自己把每一行弄成tuple
    train_data_matrix[line[1]-1,line[2]-1] = line[3]



In [6]:
del train,test,msno_set,song_set
gc.collect()

169

In [7]:
#使用sklearn的pairwise_distances计算余弦相似性。
from sklearn.metrics.pairwise import pairwise_distances as paird
user_similarity = paird(train_data_matrix,metric = 'cosine')
#x先保存一个再算一下一个

In [8]:
#下一步是预测，已经创建了相似性矩阵
#下面基于相似性举证做预测
#基于用户的评分可以根据用户均值评分
#基于产品评分缺乏针对性
def predict(ratings,similarity,type = 'user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis = 1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:,np.newaxis])
        pred = mean_user_rating[:,np.newaxis] + similarity.dot(ratings_diff)/np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity)/np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [9]:
train_user_prediction = predict(train_data_matrix,user_similarity,type = 'user')

MemoryError: 

In [ ]:
train = pd.read_csv(TRAIN)
train = train[['msno','song_id','target']]
train['user_pre_target'] = 0
for line in train.itertuples():  #自己把每一行弄成tuple
    line[4] = user_prediction[line[1]-1,line[2]-1]
#模型评价
from sklearn import metrics
from scipy import interp
cm_train = metrics.confusion_matrix(train['target'],train['user_pre_target'])
print(cm_train)

In [13]:
gc.collect()

198

In [ ]:
test = pd.read_csv(TEST)
test = test[['msno','song_id','target']]
test_data_matrix = np.zeros((n_users,n_items))
for line in test.itertuples():
    test_data_matrix[line[1]-1,line[2]-1] = line[3]
test_user_prediction = predict(test_data_matrix,user_similarity,type = 'user')

In [ ]:
#提取预测结果算概率
train = pd.read_csv(TRAIN)
test = pd.read_csv(TEST)
#member = pd.read_csv(MEMBER)
#song = pd.read_csv(SONG)
train = train[['msno','song_id','target']]
test = test[['msno','song_id','target']]
test['user_pre_target'] = 0
train['user_pre_target'] = 0
for line in train.itertuples():  #自己把每一行弄成tuple
    line[4] = user_prediction[line[1]-1,line[2]-1]
for line in test.itertuples():
    line[4] = user_prediction[line[1]-1,line[2]-1] 

In [14]:
np.savetxt(os.path.join(DATA_PATH,'user_similarity.txt'),user_similarity)
del user_similarity
gc.collect()

205

In [15]:
item_similarity = paird(train_data_matrix.T,metric = 'cosine')
np.savetxt(os.path.join(DATA_PATH,'item_similarity.txt'),item_similarity)


0

In [18]:
del item_similarity
gc.collect()

NameError: name 'item_similarity' is not defined

In [8]:
#基于用户相似性的预测
#user_similarity = np.loadtxt(os.path.join(DATA_PATH,'user_similarity.txt'))  #提取相似性文件

MemoryError: 

In [ ]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')


In [ ]:
#模型评价
from sklearn import metrics
from scipy import interp 


In [ ]:
#模型评价
from sklearn import metrics
from scipy import interp 
cm_train = metrics.confusion_matrix(y_train,model_f1.predict(X_train).round())  #训练样本的混淆矩阵
print(cm_train)
cm_test = metrics.confusion_matrix(y_test,model_f1.predict(X_test).round())  #训练样本的混淆矩阵
print(cm_test)

In [ ]:
#使用模型的混合推荐